In [1]:
import gensim
from gensim import corpora,models
from gensim.models import LdaModel
from gensim.test.utils import datapath
from gensim.parsing.preprocessing import remove_stopwords,strip_punctuation, strip_numeric,strip_short, stem_text
import pandas as pd
import unidecode
import csv
import datetime as dt

from gensim.models import ldaseqmodel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this

import matplotlib.pyplot as plt

In [2]:
filename = "amazon_reviews_us_Beauty_v1_00.tsv"
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz'

a = [] 
with open(filename) as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t' )
    for row in reader:
        a.append(row)
        break

In [3]:
fdf = pd.read_csv(filename, delimiter='\t', names=a[0], skiprows=[0])
mask = fdf.isnull().sum(axis=1) != 7
fdf = fdf[mask]
fdf["review_date"] = pd.to_datetime(fdf['review_date'],format = "%Y-%m-%d",errors='coerce')
fdf.dropna(inplace=True)
fdf["year"] = fdf["review_date"].map(lambda x: x.year)

/home/ecbm4040/miniconda3/envs/dlenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
year_count = fdf.groupby('year')['review_id'].nunique()

In [9]:
year_count = year_count.to_dict()
year_count

{2000: 33,
 2001: 257,
 2002: 567,
 2003: 1252,
 2004: 3604,
 2005: 7889,
 2006: 11601,
 2007: 28486,
 2008: 39393,
 2009: 57243,
 2010: 95308,
 2011: 180718,
 2012: 342130,
 2013: 919622,
 2014: 1604463,
 2015: 1801230}

In [6]:
# #Partition logic
# #100 random entries per year

# years = list(year_count.keys())
# years.remove(2000)
# df_list = []
# thresh = 100
# for year in years:
#     temp_df = fdf[fdf["year"] == year]
#     if (year_count[year] > thresh):
#         temp_df = temp_df.sample(thresh)
#     df_list.append(temp_df)    


# concat_dfs = []
# for i in range(5):
#     concat_dfs.append(pd.concat(df_list[3 * i: 3 * i + 3]))
# time_slice = [len(i) for i in df_list]

In [24]:
def preprocess(doc):
    return strip_short(remove_stopwords(strip_numeric(strip_punctuation(doc.lower()))),3).split()

def lda_modeling_df(df):
    corpus, dictionary = prep_corpus(df)
    lda_model = LdaModel(corpus=corpus,  # This code runs your lda
                             id2word=dictionary, 
                             random_state=100, 
                             num_topics=15,
                             passes=5,
                             chunksize=10000,
                             alpha='asymmetric',
                             decay=0.5,
                             offset=64,
                             eta=None,
                             eval_every=0,
                             iterations=100,
                             gamma_threshold=0.001,
                             per_word_topics=True)
    return lda_model, corpus, dictionary

def prep_corpus(df):
    tags = [tag for tag in df["review_body"]]
    corpus = [preprocess(tag) for tag in tags]
    dictionary = corpora.Dictionary(corpus)
    corpus = [dictionary.doc2bow(preprocess(tag)) for tag in tags]
    dictionary.filter_extremes(no_below=10, no_above=0.8)
    corpus = [dictionary.doc2bow(preprocess(tag)) for tag in tags]
    return corpus, dictionary
    
def print_lda_models(lda_model, dictionary):
    for i in range(15):
        words = lda_model.get_topic_terms(i, topn=10)
        print("Topic : " + str(i))
        for i in words:
            print("Word: " + str(dictionary[i[0]]) + "\t\t Weight: " + str(i[1])) 
        print("\n")

In [8]:
# time_slice = [len(i) for i in df_list]
# time_slice

In [9]:
# scorpus, sdictionary = prep_corpus(pd.concat(df_list))
# ldaseq = ldaseqmodel.LdaSeqModel(corpus = scorpus, id2word = sdictionary, time_slice = time_slice, num_topics=5, chunksize=1)
# temp_file = datapath("ldaseq")
# ldaseq.save(temp_file)

In [10]:
# for i in range(len(time_slice)):
#     print(ldaseq.print_topics(time = i, top_terms=5))
#     print("\n")

In [25]:
#Partition logic
#10000 random entries per year

years = list(year_count.keys())
years.remove(2000)
df_list = []
thresh = 10000
for year in years:
    temp_df = fdf[fdf["year"] == year]
    if (year_count[year] > thresh):
        temp_df = temp_df.sample(thresh)
    df_list.append(temp_df)    


concat_dfs = []
for i in range(5):
    concat_dfs.append(pd.concat(df_list[3 * i: 3 * i + 3]))
time_slice = [len(i) for i in df_list]

In [27]:
lda_model, corpus, dictionary = lda_modeling_df(concat_dfs[0])
print("Partition 1")
print("Dictionary length : " + str(len(dictionary)))
print_lda_models(lda_model, dictionary)

lda_model.log_perplexity(corpus)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
temp_file = datapath("lda_model1")
lda_model.save(temp_file)

#pyLDAvis.display(vis)
pyLDAvis.save_html(vis, 'lda1.html')

In [13]:
lda_model2, corpus2, dictionary2 = lda_modeling_df(concat_dfs[1])

print("Partition 2")
print("Dictionary length : " + str(len(dictionary2)))
print_lda_models(lda_model2, dictionary2)

lda_model2.log_perplexity(corpus2)
vis2 = pyLDAvis.gensim.prepare(lda_model2, corpus2, dictionary2)

temp_file = datapath("lda_model2")
lda_model2.save(temp_file)

#pyLDAvis.display(vis2)
pyLDAvis.save_html(vis2, 'lda2.html')

Topic : 0
Word: skin		 Weight: 0.022211274
Word: product		 Weight: 0.01663326
Word: like		 Weight: 0.014574843
Word: use		 Weight: 0.0114086205
Word: great		 Weight: 0.008800641
Word: love		 Weight: 0.008225203
Word: scent		 Weight: 0.008204887
Word: smell		 Weight: 0.0071156533
Word: products		 Weight: 0.006617378
Word: face		 Weight: 0.0064361906


Topic : 1
Word: hair		 Weight: 0.06444497
Word: product		 Weight: 0.017067049
Word: like		 Weight: 0.012170158
Word: color		 Weight: 0.010033274
Word: conditioner		 Weight: 0.008154139
Word: shampoo		 Weight: 0.008041943
Word: use		 Weight: 0.0078059034
Word: great		 Weight: 0.007312301
Word: love		 Weight: 0.006122403
Word: products		 Weight: 0.006051866


Topic : 2
Word: hair		 Weight: 0.071692534
Word: dryer		 Weight: 0.03907989
Word: iron		 Weight: 0.021595981
Word: blow		 Weight: 0.009977028
Word: heat		 Weight: 0.009841545
Word: straightener		 Weight: 0.009594052
Word: straight		 Weight: 0.009147086
Word: dry		 Weight: 0.008987663
Wo

/home/ecbm4040/miniconda3/envs/dlenv/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
lda_model3, corpus3, dictionary3 = lda_modeling_df(concat_dfs[2])

print("Partition 3")
print("Dictionary length : " + str(len(dictionary3)))
print_lda_models(lda_model3, dictionary3)

lda_model3.log_perplexity(corpus3)
vis3 = pyLDAvis.gensim.prepare(lda_model3, corpus3, dictionary3)


temp_file = datapath("lda_model3")
lda_model3.save(temp_file)

#pyLDAvis.display(vis3)
pyLDAvis.save_html(vis3, 'lda3.html')

In [ ]:
lda_model4, corpus4, dictionary4 = lda_modeling_df(concat_dfs[3])

print("Partition 4")
print("Dictionary length : " + str(len(dictionary4)))
print_lda_models(lda_model4, dictionary4)

lda_model4.log_perplexity(corpus4)
vis4 = pyLDAvis.gensim.prepare(lda_model4, corpus4, dictionary4)


temp_file = datapath("lda_model4")
lda_model4.save(temp_file)

#pyLDAvis.display(vis4)
pyLDAvis.save_html(vis4, 'lda4.html')

In [ ]:
lda_model5, corpus5, dictionary5 = lda_modeling_df(concat_dfs[4])

print("Partition 5")
print("Dictionary length : " + str(len(dictionary5)))
print_lda_models(lda_model5, dictionary5)

lda_model5.log_perplexity(corpus5)
vis5 = pyLDAvis.gensim.prepare(lda_model5, corpus5, dictionary5)


temp_file = datapath("lda_model5")
lda_model5.save(temp_file)

#pyLDAvis.display(vis5)
pyLDAvis.save_html(vis5, 'lda5.html')

In [ ]:
flda_model, fcorpus, fdictionary = lda_modeling_df(pd.concat(concat_dfs))

print("Complete Partition")
print("Dictionary length : " + str(len(fdictionary)))
print_lda_models(flda_model, fdictionary)

flda_model.log_perplexity(fcorpus)
fvis = pyLDAvis.gensim.prepare(flda_model, fcorpus, fdictionary)


temp_file = datapath("flda_model")
flda_model.save(temp_file)

#pyLDAvis.display(fvis)
pyLDAvis.save_html(fvis, 'flda.html')